In [1]:
# Achtung: Wird nur einmalig eingelesen
# bei Änderungen an tools.py muss ein Neustart des Jupyter Notebooks her
import tools    
import yfinance as yf
import pandas as pd

In [2]:
con = tools.connect_postgres("azure_postgres")

In [5]:
t = yf.Ticker("AMZN")
tools.insert_symbol(con, t)

In [7]:
t.quarterly_earnings

,Revenue,Earnings
Quarter,,
3Q2021,83360000000,20551000000
4Q2021,123945000000,34630000000
1Q2022,97278000000,25010000000
2Q2022,82959000000,19442000000


In [8]:
# Tabelle aus Ticker holen
df = t.quarterly_earnings
# Umformen, damit passend zu SQL-Tabelle
symbol = t.info["symbol"]
df["symbol"] = symbol
df["quarter"] = df.index
df["quarter"] = df["quarter"].str[2:] + "Q" + df["quarter"].str[0]
df = df.rename(columns={"Revenue":"revenue", "Earnings":"earnings"})
df
# in Datenbank hochladen
df.to_sql("earnings_quarter", con, 
    schema="finance", if_exists="append", index=False)

4

In [14]:
def insert_earnings_quarter(con, ticker):
    # Tabelle aus Ticker holen
    df = ticker.quarterly_earnings
    # Umformen, damit passend zu SQL-Tabelle
    symbol = ticker.info["symbol"]
    df["symbol"] = symbol
    df["quarter"] = df.index
    df["quarter"] = df["quarter"].str[2:] + "Q" + df["quarter"].str[0]
    df = df.rename(columns={"Revenue":"revenue", "Earnings":"earnings"})
    # in Datenbank hochladen
    df.to_sql("earnings_quarter", con, 
        schema="finance", if_exists="append", index=False)

Quarter
3Q2021    2021Q3
4Q2021    2021Q4
1Q2022    2022Q1
2Q2022    2022Q2
Name: quarter, dtype: object

In [15]:
query = f"""select quarter from finance.earnings_quarter
    where symbol = '{symbol}'"""
df_vorhanden = pd.read_sql(query,con)
df_vorhanden


,quarter
0,2021Q3
1,2021Q4
2,2022Q1


In [17]:
df[~df["quarter"].isin(df_vorhanden["quarter"])]

,revenue,earnings,symbol,quarter
Quarter,,,,
2Q2022,121234000000,-2028000000,AMZN,2022Q2


In [24]:
def insert_earnings_quarter(con, ticker):
    # Tabelle aus Ticker holen
    df = ticker.quarterly_earnings
    # Symbol in Symbol-Tabelle eintragen (falls noch nicht vorhanden)
    tools.insert_symbol(con, t)
    # Umformen, damit passend zu SQL-Tabelle
    symbol = ticker.info["symbol"]
    df["symbol"] = symbol
    df["quarter"] = df.index
    df["quarter"] = df["quarter"].str[2:] + "Q" + df["quarter"].str[0]
    df = df.rename(columns={"Revenue":"revenue", "Earnings":"earnings"})
    # schränke auf die Elemente ein, die noch nicht vorhanden sind
    query = f"""select quarter from finance.earnings_quarter
        where symbol = '{symbol}'"""
    df_vorhanden = pd.read_sql(query,con)
    df = df[~df["quarter"].isin(df_vorhanden["quarter"])]
    # in Datenbank hochladen
    df.to_sql("earnings_quarter", con, 
        schema="finance", if_exists="append", index=False)

In [20]:
insert_earnings_quarter(con, t)

In [3]:
t = yf.Ticker("TSLA")
#tools.insert_symbol(con, t)
tools.insert_earnings_quarter(con, t)

Eintrag existiert schon
